In [1]:
import os
import json

import torch
import tqdm

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
data = json.load(open("Complete_dataset/test.json"))
files = os.listdir("Complete_dataset/CT json/")
files.remove(".DS_Store")

In [3]:
files_data = {file[:-5]:json.load(open(f"Complete_dataset/CT json/{file}")) for file in files}

In [4]:
data_expanded = []
for _id, value in data.items():
    temp = {}
    temp["id"] = _id
    p_nctid = value["Primary_id"]
    s_nctid = value.get("Secondary_id")
    section_id = value["Section_id"]
    statement = value["Statement"]
    primary_evidence = files_data[p_nctid][section_id]
    temp["statement"] = statement
    temp["primary_evidence"] = primary_evidence
    # temp["label"] = value["Label"]
    
    if s_nctid is not None:
        secondary_evidence = files_data[s_nctid][section_id]
        temp["secondary_evidence"] = secondary_evidence
    
    data_expanded.append(temp)

In [5]:
def get_input_text(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment","Contradiction"])
    return f"{premise} \n Question: Does this imply that {hypothesis}? {options_}"

In [6]:
samples = []
for sample in data_expanded:
    primary_evidence = "".join([x.strip() for x in sample['primary_evidence']])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join([x.strip() for x in sample['secondary_evidence']])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text(sentence, sample['statement'])
    temp = {"text":input_text, "label":0}
    samples.append(temp)

In [7]:
from collections import Counter

In [8]:
for model_name in ['large']:#,'base','large']:
    # tokenizer = T5Tokenizer.from_pretrained(f"google/flan-t5-{model_name}",cache_dir="/mnt/data/huggingface_cache", )
    # model = T5ForConditionalGeneration.from_pretrained(f"google/flan-t5-{model_name}", cache_dir="/mnt/data/huggingface_cache", device_map="auto",torch_dtype=torch.float16)
    tokenizer = T5Tokenizer.from_pretrained(f"output_dir_{model_name}")
    model = T5ForConditionalGeneration.from_pretrained(f"output_dir_{model_name}", device_map="auto",torch_dtype=torch.float16)
    labels = []
    pred = []
    with torch.inference_mode():
        for sample in tqdm.tqdm(samples):
            labels.append(sample["label"])
            input_ids = tokenizer(sample["text"], return_tensors="pt",).input_ids.to("cuda")
            outputs = model.generate(input_ids)
            pred.append(tokenizer.decode(outputs[0]))
    
    pred = [p[5:][:-4].strip() for p in pred]
    print(Counter(pred))
    prediction_dict = {}
    for _id,pred_x in zip(data, pred):
        if pred_x.lower() == "no":
            pred_x = "Contradiction"
        elif pred_x.lower() == "yes":
            pred_x = "Entailment"
        # else:
        #     print(pred_x)
        prediction_dict[str(_id)] = {"Prediction":pred_x}
    
    json.dump(prediction_dict, open("results.json",'w'),indent=4)
    !zip "small_large/results_{model_name}.zip" results.json

  0%|          | 0/500 [00:00<?, ?it/s]/home/kamal_raj/transformers/src/transformers/generation/utils.py:1234: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 500/500 [01:26<00:00,  5.76it/s]


Counter({'Contradiction': 374, 'Entailment': 126})
updating: results.json (deflated 74%)
